# Some initial thoughts

There are a few pathways we can take to predict soil moisture:
- Predict a single soil moisture value at a specific time, e.g. 12-hours after prediction (end of day for irrigation, or 24-hours (if no irrigation today, what will it be by the time we irrigate the next day)
- Predict throughout the day at hour intervals, to identify any minimum (too dry) or maximum values (too wet)
- Predict end-of-day soil moisture for planning horizon (three days: today, tomorrow, day after)

Currently, we are pursuing the third option, which allows us to make a schedule of up to a three-day horizon.

In [1]:
import joblib
import numpy as np
import pandas as pd
from azure.cosmosdb.table.tableservice import TableService
import creds
import datetime

In [2]:
# Build data for prediction
table_service = TableService(account_name=creds.ACCNAME, account_key=creds.KEY)

In [3]:
def get_df(table):
#     now = datetime.datetime.now() - datetime.timedelta(days = 2, hours = 1)
#     datefilter = "RowKey ge '" + str(now.timestamp()) + "'"
#     weather_gen = table_service.query_entities(table, filter=datefilter)
    weather_gen = table_service.query_entities(table)
    to_df = []
    for weather in weather_gen:
        to_df.append(weather)
    df = pd.DataFrame(to_df)
    return df

In [14]:
df = get_df('SoilData').sort_values('RowKey')
df = df.reset_index(drop=True).drop(['PartitionKey','Timestamp','etag'], axis=1)
df = df.rename(columns={'humidity': 'soil_moisture'})
df['time'] = pd.to_datetime(df.RowKey.str[8:], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Phnom_Penh') # Use to get local time out of RowKey
# df['time'] = pd.to_datetime(df['time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Phnom_Penh') # Use if 'time' is a timestamp
df = df.drop('RowKey', axis=1)

In [15]:
farm_fields = df.mac_add.unique().tolist()

In [17]:
bef_df = df[['time','soil_moisture']]
days = range(1,4)
for day in days:
    prefix = str(day) + 'd'
    new_timecolumn = 'B' + prefix + 'time'
    new_soilmoisture = 'B' + prefix + 'soil_moisture'
    bef_df[new_timecolumn] = df['time'] + pd.Timedelta(str(day) + 'd')
    temp = bef_df[[new_timecolumn, 'soil_moisture']]
    temp = temp.rename({new_timecolumn: 'time', 'soil_moisture': new_soilmoisture}, axis=1)
    df = pd.merge_asof(df, temp, direction='nearest', tolerance=pd.Timedelta('1h'))

C:\Users\icarter\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [16]:
pred_df = df[['time','soil_moisture']]
days = range(1,4)
pred_labels = []
for day in days:
    prefix = str(day) + 'd'
    new_timecolumn = 'P' + prefix + 'time'
    new_soilmoisture = 'P' + prefix + 'soil_moisture'
    pred_df[new_timecolumn] = df['time'] - pd.Timedelta(str(day) + 'd')
    temp = pred_df[[new_timecolumn, 'soil_moisture']]
    temp = temp.rename({new_timecolumn: 'time', 'soil_moisture': new_soilmoisture}, axis=1)
    df = pd.merge_asof(df, temp, direction='nearest', tolerance=pd.Timedelta('1h'))
    pred_labels.append(new_soilmoisture)

C:\Users\icarter\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
df

,temperature,soil_moisture,time,mac_add,P1dsoil_moisture,P2dsoil_moisture,P3dsoil_moisture,B1dsoil_moisture,B2dsoil_moisture,B3dsoil_moisture
0,31.1,26.9,2020-07-14 16:39:56.541000+07:00,17200005,24.0,23.4,23.5,NaN,NaN,NaN
1,29.9,26.1,2020-07-14 17:31:53.495000+07:00,17200005,24.1,23.4,23.5,NaN,NaN,NaN
2,29.9,25.9,2020-07-14 17:55:01.578000+07:00,17200005,24.0,22.8,23.9,NaN,NaN,NaN
3,29.8,25.8,2020-07-14 18:12:22.268000+07:00,17200005,24.0,22.8,24.1,NaN,NaN,NaN
4,29.2,26.8,2020-07-14 18:18:09.310000+07:00,17200005,24.0,22.8,24.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
339,28.3,26.8,2020-07-20 15:05:20.807000+07:00,17200005,NaN,NaN,NaN,0.0,4.9,13.0
340,28.3,26.7,2020-07-20 15:28:28.282000+07:00,17200005,NaN,NaN,NaN,8.2,21.5,13.0
341,31.8,27.1,2020-07-20 17:12:30.412000+07:00,17200005,NaN,NaN,NaN,21.3,24.1,23.4
342,31.8,27.0,2020-07-20 17:24:04.375000+07:00,17200005,NaN,NaN,NaN,21.6,24.1,23.5


# Features worth considering

## Needed
- Duration of irrigation / amount of water irrigated (can use number of minutes irrigated that day as a variable)


## Keep in view
- Season
- Period of day (e.g. early morning, morning, afternoon, evening, night)
- Average temperature across a few hours
- Total rainfall over a period of time (e.g. in a 24-hour period)

In [19]:
def get_and_clean_cur_weather():
    table = 'WeatherAPICurrent'
    df = get_df(table).drop(['PartitionKey','RowKey','Timestamp','etag','last_updated','wind_degree'], axis=1)
    cols = df.columns.drop(['condition','localtime','wind_dir'])
    df[cols] = df[cols].apply(pd.to_numeric)
    df = df.sort_values('localtime_epoch').reset_index(drop=True)
    df['pressure_mb'] = df['pressure_mb'].astype(int)
    df['uv'] = df['uv'].astype(int)
    cols = df.columns.tolist()
    cols = cols[8:10] + cols[6:8] + cols[:6] + cols[10:]
    df = df[cols]
    df['localtime'] = pd.to_datetime(df['localtime_epoch'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Phnom_Penh')
#     df['last_updated'] = pd.to_datetime(df['last_updated_epoch'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Phnom_Penh')
    df = df.drop(['localtime_epoch','last_updated_epoch'], axis=1)
#     df['year'] = df['localtime'].dt.year
#     df['month'] = df['localtime'].dt.month
#     df['day'] = df['localtime'].dt.day
#     df['hour'] = df['localtime'].dt.hour
#     df['minute'] = df['localtime'].dt.minute
#     df['minute'] = df['minute'].map(floor)
    return df

def get_and_clean_for_weather():
    table = 'WeatherAPIForecast'
    df = get_df(table).drop(['PartitionKey','RowKey','Timestamp','etag'], axis=1)
    df.dropna(inplace=True)
    cols = df.columns.drop(['localtime','D0_condition','D1_condition','D2_condition','D0_date','D1_date','D2_date'])
    df[cols] = df[cols].apply(pd.to_numeric)
    for col in cols:
        try:
            df[col] = df[col].astype(int)
        except:
            continue
    df = df.sort_values('localtime_epoch')
    cols = df.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    df = df[cols]
    df = df.reset_index(drop=True)
    df['localtime'] = pd.to_datetime(df['localtime_epoch'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Phnom_Penh')
    df['D0_date'] = pd.to_datetime(df['D0_date_epoch'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Phnom_Penh')
    df['D1_date'] = pd.to_datetime(df['D1_date_epoch'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Phnom_Penh')
    df['D2_date'] = pd.to_datetime(df['D2_date_epoch'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Phnom_Penh')
    df = df.drop(['D0_date_epoch','D1_date_epoch','D2_date_epoch'], axis=1)
    
    keys = {}
    for day in df['D0_date'].unique():
        keys[day] = {}
        for D in ['D0_', 'D1_', 'D2_']:
            for column in [x for x in df.columns if D in x]:
                try:
                    if df[column].dtype in ['float64','int64','float32','int32']:
                        keys[day]['cur' + column + '_mean'] = df[df['D0_date'] == day][column].describe()[1]
                        keys[day]['cur' + column + '_std'] = df[df['D0_date'] == day][column].describe()[2]
                        keys[day]['cur' + column + '_states'] = len(df[df['D0_date'] == day][column].unique())
                    elif df[column].dtype in ['O', 'string']: # Sometimes this line doesnt work and gives a TypeError: data type 'string' not understood; not clear why it throws this error, hence the try/except
                        keys[day]['cur' + column + '_consensus'] = (df[df['D0_date'] == day][column].value_counts()[0] / df[df['D0_date'] == day][column].shape[0])
                        keys[day]['cur' + column + '_states'] = len(df[df['D0_date'] == day][column].unique())
                except:
                    if df[column].dtype in ['float64','int64','float32','int32']:
                        keys[day]['cur' + column + '_mean'] = df[df['D0_date'] == day][column].describe()[1]
                        keys[day]['cur' + column + '_std'] = df[df['D0_date'] == day][column].describe()[2]
                        keys[day]['cur' + column + '_states'] = len(df[df['D0_date'] == day][column].unique())
                    elif df[column].dtype in ['O']:
                        keys[day]['cur' + column + '_consensus'] = (df[df['D0_date'] == day][column].value_counts()[0] / df[df['D0_date'] == day][column].shape[0])
                        keys[day]['cur' + column + '_states'] = len(df[df['D0_date'] == day][column].unique())
    # get the days lined up in a dict

#     today_cols = [x for x in keys[df['D0_date'].unique()[0]].keys() if 'D0' in x]
    yesterday_cols = [x for x in keys[df['D0_date'].unique()[0]].keys() if 'D1' in x]
    daybefore_cols = [x for x in keys[df['D0_date'].unique()[0]].keys() if 'D2' in x]

    aligned_for1 = pd.DataFrame(columns=['D0_date'])
    aligned_for2 = pd.DataFrame(columns=yesterday_cols)
    aligned_for3 = pd.DataFrame(columns=daybefore_cols)
    aligned_for = pd.concat([aligned_for1, aligned_for2, aligned_for3])

    forecast_avg = pd.DataFrame.from_dict(keys, orient='index')

    for day in df['D0_date'].unique()[2:]:
        target_day = day
#         today = target_day
        yesterday = target_day - pd.DateOffset(1)
        daybefore = target_day - pd.DateOffset(2)

        temp = pd.Series({'D0_date': day})
        temp = temp.append(forecast_avg.loc[yesterday][yesterday_cols])
        temp = temp.append(forecast_avg.loc[daybefore][daybefore_cols])

        aligned_for = aligned_for.append(temp, ignore_index=True)
        
    del temp
    
    aligned_for['D0_date'] = pd.to_datetime(aligned_for['D0_date'])
        
    df = pd.merge(df, aligned_for, on='D0_date')

    return df

def get_all_clean_weather():
    left = get_and_clean_cur_weather()
#     print('%s rows in cur' % left.shape[0])
    right = get_and_clean_for_weather()
#     print('%s rows in for' % right.shape[0])
    df = pd.merge_asof(left, right, direction='nearest') # This will get the nearest forecast to the current weather, whether taking place before or after, without any time constraint
#     df = pd.merge_asof(left, right, direction='nearest', tolerance=pd.Timedelta('120s')) # This will get the nearest forecast to the current weather, whether taking place before or after, and within 2 minutes
#     df = pd.merge_asof(left, right, direction='backward') # This will get the latest forecast up to the current time, but not after, regardless of how long away

    # Running 'backward' creates a gaps of 20 minutes or longer between cur and for associations, as the forecast seems to take place after the current weather is posted to the API. therefore prefer to use nearest for matching, as we can run the model a few moments after the forecast comes in to make the prediction.
    
#     print('%s rows before drop' % df.shape[0])
#     df.dropna(inplace=True)
#     print('%s rows after drop' % df.shape[0])
    return df

def get_conditions_set():
    weather_gen = table_service.query_entities('WeatherAPICurrent', select='condition')
    to_df = []
    for weather in weather_gen:
        to_df.append(weather)
    curdf = pd.DataFrame(to_df).drop('etag', axis=1)
    weather_gen = table_service.query_entities('WeatherAPIForecast', select="D0_condition, D1_condition, D2_condition")
    to_df = []
    for weather in weather_gen:
        to_df.append(weather)
    fordf = pd.DataFrame(to_df).drop('etag', axis=1)
    conditions_set = set()
    for x in curdf.condition.unique():
        conditions_set.add(x)
    for col in ['D0_condition', 'D1_condition', 'D2_condition']:
        for x in fordf[col].unique():
            conditions_set.add(x)
    return conditions_set

def get_wind_dir_set():
    weather_gen = table_service.query_entities('WeatherAPICurrent', select='wind_dir')
    to_df = []
    for weather in weather_gen:
        to_df.append(weather)
    curdf = pd.DataFrame(to_df).drop('etag', axis=1)
    conditions_set = set()
    for x in curdf.wind_dir.unique():
        conditions_set.add(x)
    return conditions_set

def cleaned_data():
    df = get_all_clean_weather().dropna(subset=['localtime_epoch']).reset_index()
    
    df.loc[df['precip_mm'] > 0, 'is_raining'] = 1
    df['is_raining'] = df['is_raining'].fillna(0)
    
    drop_columns = ['index','D0_date','D1_date','D2_date']
    df = df.drop(drop_columns, axis=1)
    
    for col in ['condition','D0_condition', 'D1_condition', 'D2_condition']:
        for condition in get_conditions_set():
            new_condition = col + '_' + condition.replace(' ', '_') # To satisfy the Table Service requirement that column names do not have spaces
            df.loc[df[col] == condition, new_condition] = int(1)
            df[new_condition] = df[new_condition].fillna(0)
        df.drop(col, axis=1, inplace=True)

    for wind in get_wind_dir_set():
        new_wind_dir = 'wind_dir_' + wind
        df.loc[df['wind_dir'] == wind, new_wind_dir] = int(1)
        df[new_wind_dir] = df[new_wind_dir].fillna(0)
    df.drop('wind_dir', axis=1, inplace=True)
    df.dropna(inplace=True)
    return df

In [20]:
weather_df = cleaned_data()

In [21]:
weather_df = weather_df.rename(columns={'localtime':'time'})

In [22]:
df = pd.merge_asof(df, weather_df, direction='backward')

In [23]:
df

,temperature,soil_moisture,time,mac_add,P1dsoil_moisture,P2dsoil_moisture,P3dsoil_moisture,B1dsoil_moisture,B2dsoil_moisture,B3dsoil_moisture,...,wind_dir_SSE,wind_dir_N,wind_dir_NW,wind_dir_SSW,wind_dir_NNW,wind_dir_ESE,wind_dir_ENE,wind_dir_WSW,wind_dir_S,wind_dir_W
0,31.1,26.9,2020-07-14 16:39:56.541000+07:00,17200005,24.0,23.4,23.5,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,29.9,26.1,2020-07-14 17:31:53.495000+07:00,17200005,24.1,23.4,23.5,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,29.9,25.9,2020-07-14 17:55:01.578000+07:00,17200005,24.0,22.8,23.9,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,29.8,25.8,2020-07-14 18:12:22.268000+07:00,17200005,24.0,22.8,24.1,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,29.2,26.8,2020-07-14 18:18:09.310000+07:00,17200005,24.0,22.8,24.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,28.3,26.8,2020-07-20 15:05:20.807000+07:00,17200005,NaN,NaN,NaN,0.0,4.9,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
340,28.3,26.7,2020-07-20 15:28:28.282000+07:00,17200005,NaN,NaN,NaN,8.2,21.5,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
341,31.8,27.1,2020-07-20 17:12:30.412000+07:00,17200005,NaN,NaN,NaN,21.3,24.1,23.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
342,31.8,27.0,2020-07-20 17:24:04.375000+07:00,17200005,NaN,NaN,NaN,21.6,24.1,23.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [25]:
df.dropna()

,temperature,soil_moisture,time,mac_add,P1dsoil_moisture,P2dsoil_moisture,P3dsoil_moisture,B1dsoil_moisture,B2dsoil_moisture,B3dsoil_moisture,...,wind_dir_SSE,wind_dir_N,wind_dir_NW,wind_dir_SSW,wind_dir_NNW,wind_dir_ESE,wind_dir_ENE,wind_dir_WSW,wind_dir_S,wind_dir_W
138,28.7,6.3,2020-07-17 15:42:32.658000+07:00,17200005,21.5,10.4,26.7,0.0,22.9,26.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
139,30.6,23.5,2020-07-17 16:45:47.703000+07:00,17200005,21.6,21.1,27.1,23.4,24.0,26.9,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
140,30.3,23.4,2020-07-17 16:51:31.888000+07:00,17200005,21.6,21.1,27.1,23.4,24.2,26.9,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
141,30.6,23.4,2020-07-17 17:08:52.861000+07:00,17200005,24.1,21.3,27.1,23.4,24.2,26.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
142,31.4,23.5,2020-07-17 17:26:09.642000+07:00,17200005,24.1,21.6,27.0,23.4,24.1,26.1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
144,33.9,24.1,2020-07-17 18:11:58.681000+07:00,17200005,23.6,21.8,26.7,22.8,24.0,25.8,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
145,33.4,24.0,2020-07-17 18:17:39.999000+07:00,17200005,23.6,21.8,26.7,22.8,24.0,26.8,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
146,33.2,24.0,2020-07-17 18:40:43.604000+07:00,17200005,23.7,21.5,26.7,22.8,24.4,22.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,32.9,23.9,2020-07-17 18:52:20.723000+07:00,17200005,23.7,21.3,26.7,22.8,24.4,22.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
